In [4]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [5]:
# query new data <= 1/1
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W46`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W47`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W48`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W49`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W50`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W51`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W52`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W53`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W01`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W02`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W03`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.2021-W04`
"""

In [6]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [7]:
data_in_use.sort_values(by=['Week']).tail()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
50123,2021-01-24 07:46:47+00:00,2021-01-24 07:49:01+00:00,Production,0016 of 0017,D2,None,None,KL7,Monday Assembly,Sunday Morning,iXUsr_Siregar(6092),2,2.23,2021-04
50134,2021-01-24 06:58:05+00:00,2021-01-24 07:02:29+00:00,Production,0010 of 0017,D2,None,None,KL7,Monday Assembly,Sunday Morning,iXUsr_Siregar(6092),2,4.40,2021-04
19758,2021-01-22 19:50:56+00:00,2021-01-22 19:55:28+00:00,Production,0001 of 0008,D2,None,None,KL7,Thursday Assembly,Friday Afternoon,iXUsr_Wang(9965),3,4.53,2021-04
19685,2021-01-22 14:33:16+00:00,2021-01-22 14:36:35+00:00,Production,0002 of 0016,H2,None,None,KL8,Saturday Assembly,Friday Afternoon,iXUsr_Wang(9965),3,3.32,2021-04
37729,2021-01-23 14:22:44+00:00,2021-01-23 14:22:53+00:00,Preparation/Changeover,None,E4,None,None,KL5,None,Saturday Afternoon,iXUsr_Selina(6751),3,0.15,2021-04


In [8]:
data_in_use.shape

(65021, 14)